In [ ]:
import sys
sys.path.append(r'../..')

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib
matplotlib.rcParams['figure.figsize'] = [10, 4]
matplotlib.rcParams['lines.linewidth'] = 2
import numpy as np
from numpy import pi
from scipy.linalg import cholesky, eigh
from composites import isotropic_plate

from tudaesasII.quad4r import Quad4R, update_K, update_M, DOF

def mag2db(mag):
    return 20*np.log10(mag)

nx = 25
ny = 25

a = 0.300 # [m]
b = 0.274 # [m]

E = 70.e9 # Pa
nu = 0.33
rho = 2.7e3 # kg/m3
h = 0.001 # m

xtmp = np.linspace(0, a, nx)
ytmp = np.linspace(0, b, ny)
xmesh, ymesh = np.meshgrid(xtmp, ytmp)
ncoords = np.vstack((xmesh.T.flatten(), ymesh.T.flatten())).T
x = ncoords[:, 0]
y = ncoords[:, 1]

nids = 1 + np.arange(ncoords.shape[0])
nid_pos = dict(zip(nids, np.arange(len(nids))))
nids_mesh = nids.reshape(nx, ny)
n1s = nids_mesh[:-1, :-1].flatten()
n2s = nids_mesh[1:, :-1].flatten()
n3s = nids_mesh[1:, 1:].flatten()
n4s = nids_mesh[:-1, 1:].flatten()

plate = isotropic_plate(thickness=h, E=E, nu=nu, calc_scf=True)

N = DOF*nx*ny
K = np.zeros((N, N))
M = np.zeros((N, N))
quads = []

for n1, n2, n3, n4 in zip(n1s, n2s, n3s, n4s):
    pos1 = nid_pos[n1]
    pos2 = nid_pos[n2]
    pos3 = nid_pos[n3]
    pos4 = nid_pos[n4]
    r1 = ncoords[pos1]
    r2 = ncoords[pos2]
    r3 = ncoords[pos3]
    normal = np.cross(r2 - r1, r3 - r2)
    assert normal > 0 # guaranteeing that all elements have CCW positive normal
    quad = Quad4R()
    quad.rho = rho
    quad.n1 = n1
    quad.n2 = n2
    quad.n3 = n3
    quad.n4 = n4
    quad.scf13 = plate.scf_k13
    quad.scf23 = plate.scf_k23
    quad.h = h
    quad.ABDE = plate.ABDE
    update_K(quad, nid_pos, ncoords, K)
    update_M(quad, nid_pos, ncoords, M)
    quads.append(quad)

bk = np.zeros(N, dtype=bool) # constrained DOFs, can be used to prescribe displacements
# eliminating u and v
bk[0::DOF] = True
bk[1::DOF] = True

# unknown DOFs
bu = ~bk

# sub-matrices corresponding to unknown DOFs
Kuu = K[bu, :][:, bu]
Muu = M[bu, :][:, bu]

L = cholesky(Muu, lower=True)
Linv = np.linalg.inv(L)
Ktilde = Linv @ Kuu @ Linv.T

rbmodes = 3
Nmodes = 40
gamma, V = eigh(Ktilde, eigvals=(0, Nmodes-1)) # already gives V[:, i] normalized to 1
V = V[:, rbmodes:]
gamma = gamma[rbmodes:]

omegan = gamma**0.5

P = V


zeta = 0.02

omegad = omegan*np.sqrt(1 - zeta**2)

print('omegan [Hz]', omegan/(2*pi))
print('omegad [Hz]', omegad/(2*pi))


Hammer Impact Parameters
---

In [ ]:
tmax = 2
time_steps = 2**16

t = np.linspace(0, tmax, time_steps)
dt = t[1] - t[0]

xpos = np.linspace(0, a, 5)
ypos = np.linspace(0, b, 5)
xpos, ypos = np.meshgrid(xpos, ypos)
xpos = xpos.flatten()
ypos = ypos.flatten()
points = dict(zip(np.arange(1, 26), np.vstack((xpos, ypos)).T))
usedpoints = list(points.keys())

hammer = np.zeros_like(t)
impulse = 5*0.003
print('impulse', impulse)
hammer[:2] = -impulse/dt

Simulating Impact Hammer Test
---

In [ ]:
for mode in range(10):
    plt.figure()
    Uu = (Linv.T @ V[:, mode])
    U = np.zeros(K.shape[0])
    U[bu] = Uu
    ax = plt.gca()
    cs = ax.contour(xmesh, ymesh, U[2::DOF].reshape(xmesh.shape).T, levels=5)
    ax.clabel(cs, inline=1, fontsize=10)
    plt.gca().set_aspect('equal')
    print('omega1', omegan[mode]/(2*pi))

    acc_pos = np.where(np.isclose(x, a) & np.isclose(y, 0))[0][0]
    plt.plot(x[acc_pos], y[acc_pos], 'ro', ms=30)
    plt.plot(points[1][0], points[1][0], 'gs', ms=30)

In [ ]:
Nsamples = t.size
print('Nsamples', Nsamples)

# dynamic analysis

# initial conditions
v0 = np.zeros(N)
u0 = np.zeros(N)

# modal space
r0 = P.T @ L.T @ u0[bu]
rdot0 = P.T @ L.T @ v0[bu]
phi = np.zeros(Nmodes-rbmodes)
check = r0 != 0
phi[check] = np.arctan(omegad[check]*r0[check]/(zeta*omegan[check]*r0[check] + rdot0[check]))
A0 = np.sqrt(r0**2 + (zeta*omegan/omegad*r0 + rdot0/omegad)**2)

on = omegan[:, None]
od = omegad[:, None]

# convolution integral: general load as a sequence of impulse loads
def r_t(t, t1, t2, on, zeta, od, fmodaln):
    tn = (t1 + t2)/2
    # damped function
    H = np.heaviside(t - tn, 1.)
    h = np.zeros((Nmodes-rbmodes, t.shape[0]))
    check = t >= tn
    h[:, check] = 1/od*np.exp(-zeta*on*(t[check] - tn))*np.sin(od*(t[check] - tn))*H[check]
    return fmodaln*dt*h

# homogeneous solution
rh = A0[:, None]*np.exp(-zeta*on*t)*np.sin(od*t + phi[:, None])

f = np.zeros(N)

responses = {}
u = np.zeros((N, len(t)))
for point in usedpoints:
    ximpact, yimpact = points[point]
    print('analysing impact at point %d:' % point, ximpact, yimpact)
    force_pos = np.where(np.isclose(x, ximpact) & np.isclose(y, yimpact))[0][0]
    
    f[:] = 0
    f[DOF*force_pos+2] += hammer[0]
       
    # calculating modal forces
    fmodaln = (P.T @ Linv @ f[bu])[:, None]

    # convolution
    rpc = r_t(t, 0, 0, on, zeta, od, fmodaln)

    # superposition between homogeneous solution and forced solution
    r = rh + rpc

    # transforming from r-space to displacement
    u[bu] = Linv.T @ P @ r
    responses[point] = u[DOF*acc_pos+2].copy()


Checking signals in time domain
---

In [ ]:
for point in usedpoints:
    plt.figure(figsize=(5, 3))
    plt.plot(t, responses[point])
    plt.xlabel('time [s]')
    plt.ylabel('displacement [m]')
    plt.ylim(-0.004, 0.004)

Experimentally Determining Mode Shapes
---

In [ ]:
H_points ={}
dt = t[1] - t[0]
freqs = np.fft.rfftfreq(Nsamples, dt)
xf = np.fft.rfft(hammer)/(Nsamples//2)
for point in usedpoints:
    yf = np.fft.rfft(responses[point])/(Nsamples//2)
    pxx = np.conj(xf)*xf 
    pxy = np.conj(xf)*yf 
    H = np.zeros_like(pxx)
    check = ~np.isclose(pxx, 0+0j)
    H[check] = pxy[check]/pxx[check]
    H_points[point] = H

In [ ]:
plt.figure(figsize=(10, 6))
for point in [1, 3, 5, 11, 13, 15, 21, 23, 25]:
    H = H_points[point]
    plt.plot(freqs, H.imag)
    plt.xlim(35, 80)
    plt.ylim(-0.007, 0.007)
    plt.title('$H(i\omega)$')
    plt.xlabel('Hz')
    plt.ylabel('[m / N]')

    
plt.figure(figsize=(7, 5))
for point in usedpoints:
    H = H_points[point]
    plt.semilogy(freqs, np.abs(H))
    plt.xlim(0, 3000)
    plt.ylim(-0.007, 0.007)
    plt.title('$H(i\omega)$')
    plt.xlabel('Hz')
    plt.ylabel('[m / N]')
plt.xlim(0, 300)


plt.figure(figsize=(7, 5))
H = H_points[1]
plt.semilogy(freqs, np.abs(H))
plt.xlim(0, 300)
plt.ylim(-0.007, 0.007)
plt.title('$H(i\omega)$')
plt.xlabel('Hz')
plt.ylabel('[m / N]')
plt.xlim(0, 300)

In [ ]:
modeshape = []
for point in usedpoints:
    H = H_points[point]
    plt.figure(figsize=(5, 3))
    plt.plot(freqs, H.imag)
    plt.xlim(35, 80)
    plt.ylim(-0.007, 0.007)
    plt.title('$H(i\omega)$')
    plt.xlabel('Hz')
    plt.ylabel('[m / N]')
    ximpact, yimpact = points[point]
    modeshape.append([point, ximpact, yimpact, H.imag[np.isclose(freqs, omegan[0]/2/pi, rtol=0.01)][0]])

modeshape = np.asarray(modeshape)


In [ ]:
point, ximpact, yimpact, ampl = modeshape.T
plt.contourf(ximpact.reshape(5, 5), yimpact.reshape(5, 5), np.asarray(ampl).reshape(5, 5), levels=10)
for xp, yp, pt in zip(ximpact, yimpact, point):
    plt.text(xp, yp, s=str(int(pt)), fontsize=14)
plt.gca().set_aspect('equal')
plt.colorbar()

In [ ]:
mode = 0
Uu = (Linv.T @ V[:, mode])
U = np.zeros(K.shape[0])
U[bu] = Uu
plt.contourf(xmesh, ymesh, U[2::DOF].reshape(xmesh.shape).T, levels=100)
plt.gca().set_aspect('equal')
plt.colorbar()